# 1. Access and analyze seismograms

- Name:
- First name:
- Student ID:
- Email:

<img width=150px src="https://www.ipgp.fr/sites/default/files/ipgp_up_verticaux_rvb.png" />

## Preamble

This Jupyter notebook is part of the _Scientific Computing for Geophysical Problems_ class. I will collect it __at the end of this class__. Please, provide an answer to the questions within Markdown or code cells when needed. Once complete, please email me your notebook in PDF format at seydoux@ipgp.fr, with the filename `lastname_firstname_1.ipynb` all lowercase.

This notebook shows how to __download, plot, and analyze seismic waveforms in time and frequency__. It goes from identifying events of interest by exploring online resources with reported events to extracting the seismic waveforms from suited data centers and analyzing their properties. 

- [Question 1. Waveform query.](#question-1-waveform-query)
- [Question 2. Stream and trace metadata.](#question-2-stream-and-trace-metadata)
- [Question 3. Analysis of the raw waveform.](#question-3-analysis-of-the-raw-waveform)
- [Question 4. Saving and reading waveforms locally.](#question-4-saving-and-reading-waveforms-locally)
- [Question 5. Instrument response correction.](#question-5-instrument-response-correction)
- [Question 6. Spectral analysis.](#question-6-spectral-analysis)
- [Question 7. Filtering.](#question-7-filtering)
- [Question 8. Spectro-temporal analysis.](#question-8-spectro-temporal-analysis)
- [Question 9. Bonus analysis with a selected event.](#question-9-bonus-analysis-with-a-selected-event)

## Requirements

This notebook relies on the four following Python libraries; please ensure that the virtual Python environment you are using includes them correctly:
- [ObsPy](https://github.com/obspy/obspy/wiki), an open-source project that provides a Python framework for processing seismological data. It provides parsers for standard file formats, clients to access data centers, and signal processing routines that allow the manipulation of seismological time series.
- [NumPy](https://numpy.org), an open-source project aiming to enable numerical computing with Python.
- [SciPy](https://numpy.org), fundamental algorithms for scientific computing in Python.
- [Matplotlib](https://matplotlib.org), a comprehensive library for creating static, animated, and interactive visualizations in Python.

If unavailable on your machine, the following steps will guide you towards correctly installing the packages mentioned above. [Anaconda](https://www.anaconda.com) is a Python package manager that allows you to install Python libraries easily and is available from the virtual machine. If you wish to install it on your machine locally, please follow the instructions at https://www.anaconda.com. The following steps create a new environment named `scientific_computing` and install the latest version of Python 3.10 with ObsPy and Matplotlib. Note that the library NumPy is automatically installed with ObsPy. This will allow you to run this notebook properly.

Once these steps are successfully achieved, you can run the next cell to check that the installation is correct. The magic command "% matplotlitb widgets" ensures that the plots will be rendered interactively. It should appear before importing the matplotlib library.

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import obspy 
import scipy

## Searching datasets of interest

__Requesting seismic data first requires a target event of interest.__ This stage is generally motivated by a scientific question or by curiosity to look at some recent events. Here we deliver the sources where this information is accessible. You may also find crucial information from other regional and national earthquake information services (volcano observatories, glacial area observatories).

- The [United States Geological Survey (USGS) Advanced National Seismic System Comprehensive Catalog of Earthquakes Events and Products](https://earthquake.usgs.gov/earthquakes/search) provides one of the most comprehensive global earthquake catalogs. It delivers earthquake source parameters (e.g. hypocenters, magnitudes, phase picks, and amplitudes) and other products (e.g. moment tensor solutions, macroseismic information, tectonic summaries, maps) produced by contributing seismic networks. The webpage https://earthquake.usgs.gov/data/comcat/ provides a list of contributing seismic networks.
- The [ISC-GEM Global Instrumental Earthquake Catalogue](http://www.isc.ac.uk/iscbulletin/search/catalogue/) (1904-2018) is the result of a special effort to adapt and substantially extend and improve currently existing bulletin data for large earthquakes (magnitude 5.5 and above, plus continental events down to magnitude 5.0) to serve the requirements of the specific user group who assess and model seismic hazard and risk.
- The [European-Mediterranean Seismological Centre (EMSC), France](https://www.emsc-csem.org) was founded in 1975, following a recommendation from the European Seismological Commission (ESC). The ESC is a regional commission of the International Association of Seismology and Physics of the Earth's Interior (IASPEI), itself a specialized association of the International Union of Geodesy and Geophysics (IUGG).

__Seismic stations or networks__ are also listed in several websites. The [International Federation of Digital Seismograph Networks (FDSN)](https://www.fdsn.org/about/) goals related to station siting and instrumentation are to provide stations with good geographic distribution, recording data with 24 bits of resolution in continuous time series with at least a 20 sample per second sampling rate. You can find a [list of seismic networks with links to station maps and descriptions](https://www.fdsn.org/networks/) online to fetch seismic data on specific regions and temporal coverage. Also, the [Incorporated Research Institutions for Seismology (IRIS)](https://ds.iris.edu/gmap/) is a consortium of universities dedicated to the operation of science facilities for the acquisition, management, and distribution of seismological data. IRIS delivers seismic waveforms and metadata based on the FDSN inventory, which facilitates the [search of the temporary or permanent seismic network](https://ds.iris.edu/gmap/). Here are some famous ones:

- [IU, the Global Seismograph Network (GSN) operated by IRIS and USGS](https://www.fdsn.org/networks/detail/IU/), described by a global distribution of high-gain, co-located broadband velocity instruments and accelerometers.
- [II, the Global Seismograph Network (GNS) operated by IRIS and IDA](https://www.fdsn.org/networks/detail/II/).
- [G, GEOSCOPE](https://www.fdsn.org/networks/detail/G/) is a global seismological network created in 1982 to respond to the challenge of instrumenting isolated sites all around the world. More information is available at the [Institut de Physique du Globe de Paris website](http://geoscope.ipgp.fr/index.php/en/stations/station-map).
- [FR: RESIF](https://www.fdsn.org/networks/detail/FR/) and other broadband and permanent accelerometric networks in metropolitan France allow monitoring of the seismicity in metropolitan France and the development of seismic risk and hazards maps.
- [PF: Piton de la Fournaise Volcano Observatory Network (Reunion Island)](https://www.fdsn.org/networks/detail/PF/) allows monitoring of the volcano-seismic activity of the Piton de la Fournaise volcano. 
- [GL: Guadeloupe Seismic and Volcano Observatory Network](https://www.fdsn.org/networks/detail/GL/) allows monitoring of the volcano-seismic activity of La Soufrière volcano. 
- [HV: Hawaiian Volcano Observatory Network](https://www.fdsn.org/networks/detail/HV/) allows monitoring of the volcano-seismic activity in Hawaii. 
- [TA: USArray Transportable Array](https://www.fdsn.org/networks/detail/TA/) is a transportable array that was deployed between 2004 and 2019 in the US.

__Finally, a comprehensive list of datacenters__ that support the FDSN webservices can be found at https://www.fdsn.org/webservices/datacenters/. Among others, you can find here a shortlist of international and French data centers:
- [IRIS Data Management Center](https://ds.iris.edu/ds/nodes/dmc/)
- [EIDA: European Integrated Data Archive](http://www.orfeus-eu.org/data/eida/)
- [RESIF: French seismological and geodetic network](http://seismology.resif.fr)
- [IPGP: data center of the Institut de Physique du Globe de Paris](http://datacenter.ipgp.fr/data.php)

## Question 1. Waveform query

Several tools allow downloading seismic waveforms provided by the datacenters via the [FDSN Web Services](https://www.fdsn.org/webservices/) which is [natively included in the obspy Python library](https://docs.obspy.org/packages/obspy.clients.fdsn.html). Here we show an example of how to download seismic data from the IRIS datacenter with the `Client` interface of `obspy`. Once the interface is connected to a client of interest, we can request a seismic waveform by specifying several metadata such as the network code, the station code, the location of the sensor in the station, the channel name, the start and end datetimes, and other information. Please check the [documentation of the `get_waveform` method](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html).

Seismic records are organized in a specific type of object in Python called a `trace`. Since several traces can be downloaded at once, several traces are gathered into a `stream` object. By default, the `get_waveform` method returns a `stream` event if only one trace was downloaded. You can print a succinct amount of information about the stream with the `print` function.

> 1. What kind of seismic network is the IU network? Where is the seismic station CTAO located? 
> 1. From the date and time used in the above example waveform query, which earthquake was recorded by the seismic sensor CTAO? 
> 1. What is the approximate epicentral distance of the earthquake to the seismic station? Please use the different websites provided in the previous section together with the obspy documentation to answer that question. You can also use the obspy method [`location2degrees`](https://docs.obspy.org/packages/autogen/obspy.geodetics.base.locations2degrees.html) to answer the last question.

In [ ]:
from obspy.clients.fdsn import Client

client = Client("IRIS")

stream = client.get_waveforms(
    network="IU",
    station="CTAO",
    location="*",
    channel="BHZ",
    starttime=obspy.UTCDateTime("2021-03-04T19:28:33"),
    endtime=obspy.UTCDateTime("2021-03-04T21:30:00"),
    attach_response=True,
)

print(stream)

## Question 2. Stream and trace metadata

A `stream` can be seen as a list of `trace` objects, plus methods. More precisely, almost all of the methods available in a `trace` object is available in a `stream` object, to be applied on all traces individually. In the following examples, we will narrow the focus on the first trace of the downloaded stream, and take a look at its metadata. A trace's metadata are stored in a dictionnary named `stats`. The entries of this dictionnary may depend on the format of the seismic trace. The most common seismic formats are `SAC` and `MSEED` (visible in the `_format` entry of the `stats` dictionnary).

> 1. take time to discover the `stats` dictionnary of a selected `trace` and understand the different entries. More information about this dictionnary can be found in the [obspy documentation](https://docs.obspy.org/packages/autogen/obspy.core.trace.Stats.html). In particular, what are the `npts`, `delta` and `sampling_rate` entries? Please define everything with respect to the digital signal $u[n] = u(n\Delta t)$ and other quantities seen during the class. 
> 1. In which format was the trace downloaded? Considering the online [documentation about the data formats on IRIS](http://ds.iris.edu/ds/nodes/dmc/data/formats/), what does `number_of_records` mean in that case? 
> 1. What information is presented in the `response` entry? What is the signification of a sensor sensitivity? 

## Question 3. Analysis of the raw waveform

> 1. In the following example, we define a time vector in number of samples $t = 0\ldots N$. Modify the cell below to turn this time vectors into a relative time (in seconds) from the begining of the record based on the entries of the `stats` dictionary, and update the x-axis name accordingly. 
> 1. How would you proceed to use an absolute time array with date ticks? 
> 1. What is the type and unit of the raw `data`? Please name the axis accordingly, and explain your answer. What would you need to turn the waveform into physical units?

In [ ]:
# Select a seismic trace
trace = stream[1]

# Waveform data
waveform = trace.data

# (Question 3) Turn the following vector into times in seconds
time = np.arange(trace.stats.npts)

# Plot
plt.figure(1)
plt.clf()
plt.plot(time, waveform)

# (Question 3) name the axes correctly
plt.xlabel("Give me a label", color="red")
plt.ylabel("Give me a label", color="red")

# Show
plt.title(trace.id)
plt.grid()
plt.show()

## Question 4. Saving and reading waveforms locally

Of course, in order to avoid re-downloading the waveforms everytime, one should be able to write it down to a local or remote disk. This can be achieved at the `stream` or the `trace` level. Here again, you can decide the format you would like to use. You can look at the specificities of every format in the documentation. Then, local waveforms can be loaded directly in with the `obspy.read` method. This method allows to read almost all existing seismic data formats. Once the previous cell was ran, you can execute the next cell to read back the data.

> 1. Use the code below to save the seismogram locally.
> 1. Change the format between SAC and MSEED. What main differences exist between these two formats?
> 1. Load the locally-saved stream from MSEED format. What missing in the trace metadata after reading?

In [ ]:
stream.write(filename="seismogram.sac")

# Read the saved stream below into a new stream and explore its metadata (compare with the original stream)

## Question 5. Instrument response correction

For a physical interpretation of the waveform amplitude, we need to convert the raw waveforms into physical units such as ground displacement, velocity, or acceleration. Because we asked the program to _attach the response_ in the `get_waveform` method, we can use the `trace.remove_response()` method to remove the instrument response. Please check the documentation to understand how this method works.

> 1. Why do we need for such a complete tool to remove the instrument response? 
> 1. Why don't we simply infer how many velocity is 1 count, and divide the counts by this value?
> 1. Add code to the cell below to show the corrected waveform. What changed compared with the raw waveform? 
> 1. What is approximately the period of the very-low frquency signal?
> 1. We can use a `pre_filt` tuple that defines a bandpass filter in frequency domain to the data before deconvolution. The list or tuple defines the four corner frequencies (f1, f2, f3, f4) of a cosine taper which is one between f2 and f3 and tapers to zero for f1 < f < f2 and f3 < f < f4. Can you use this filter to remove the very-low-frequency signature?
> 1. Plot the displacement $u[n]$, velocity $\dot u[n]$, and acceleration $\ddot u[n]$ on three separate subplots. What are the main differences between the signals?

In [ ]:
# Keep track of the original trace, work on a copy
trace_physical = trace.copy()

# Prefilter, remove response and get physical units
trace_physical.remove_response(output="VEL")

# Perform the plot below

## Question 6. Spectral analysis

The time-domain representation of a waveform gives information about the temporal evolution of the rupture, the different kinds of seismic phases recorded, etc. Another critical tool of the seismologist is to observe the waveform's properties as a function of frequency. The Fourier transform is the most robust and stable tool to calculate the waveform's spectrum. Note that many other transforms allow visualizing the spectral properties of signals against frequencies. Other methods exist because of the robustness to noise, capability to be applied on very-short duration time series, or other specificities.

> 1. Using the `numpy.fft` module, calculate the Fourier spectrum $\tilde u(f)$ of the seismic trace $u[n]$ in the cell below.
> 1. Withing several subplots, show the raw, displacement, velocity, and acceleration spectrum $|\tilde u(f)|$  in a log-log scale.
> 1. What is the effect of the pre-filter in every case?
> 1. Give a very simple interpretation of the observed spectra: where is the energy located for each physical unit?

In [ ]:
# Get the Fourier spectrum of the trace
spectrum = 1

# Define the frequency vector accordingly
frequencies = 1

# Show
plt.figure(4)
plt.clf()
plt.loglog(frequencies, np.abs(spectrum))

# Of course, add labels, titles, etc. -- You know, the routine thing.
plt.text(1, 1, "An amazing spectrum should go here.", ha="center", color="green")

# Show
plt.show()

## Question 7. Filtering

Thanks to digital processing tools, it is possible to filter out the waveform content within specific frequency bands. The cell below shows how to filter the seismic signal in a band of interest.

> 1. Show the filtered trace in both time and frequency domains. 
> 1. What is the effect of the filter? 
> 1. Using a low-pass filter below 0.1 Hz, what features of the waveform do you better highlight? And with a high-pass filter above 2 Hz?
> 1. By playing with the `corner` value in the filter above, how do you better select the frequencies of interest?

In [ ]:
trace_filter = trace_physical.copy()
trace_filter.filter("bandpass", freqmin=1, freqmax=3, corners=4)

# Plot, labels, art, and analysis.

## Question 8. Spectro-temporal analysis

> 1. The following cell performs a spectrogram —a time-dependant Fourier spectrum analysis $|\tilde u(f, t)|$— of the raw trace.
> 1. What are the `nperseg` and the `noverlap` parameters useful for?
> 1. Describe at least four different signal patterns seen in the spectrogram, depending on their duration, frequency bandwidth, etc.
> 1. Two waves exhibit a typical signature in the spectrogram, with an increasing frequency against time. What does this imply for the speed of the seismic waves?
> 1. How does this spectrogram changes with respect to the input waveform units?

In [ ]:
from scipy.signal import spectrogram

# Extract waveform
waveform = trace.data
times = trace.times()

# Calculate spectrogram
frequencies, spectrogram_times, psd = spectrogram(waveform, fs=trace.stats.sampling_rate, nperseg=4000, noverlap=2500)

# This is because the first frequency is set to 0
frequencies[0] = frequencies[1]

# Plot
plt.figure(6)
plt.clf()
_, axes = plt.subplots(nrows=2, num=6, sharex=True)

axes[0].plot(times, waveform)
axes[1].pcolormesh(spectrogram_times, frequencies, np.log(np.abs(psd)))
axes[1].set_yscale("log")

# Do a crazy job with labels

# Show
plt.show()


## Question 9. Bonus: analysis with a selected event.

> The time has come. Re-run the entire notebook with a event of your choice. Motivate your choice and anwser most of the questions with the new event.